<a href="https://colab.research.google.com/github/eng-accelerator/Submissions-C4/blob/main/Jeyalakshmi_Subbiah/Breakout_Assignment_Gradio_day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies (run once)
!pip -q install -U gradio transformers accelerate sentencepiece huggingface_hub


In [ ]:
import os
import json
import time
from datetime import datetime

import gradio as gr
from huggingface_hub import login
from transformers import pipeline


In [1]:
# Read token from environment
from huggingface_hub import whoami
from google.colab import userdata

# Get your Hugging Face token from Colab Secrets
hf_token = userdata.get('HF_TOKEN')

# Verify the token by checking your identity
try:
    user_info = whoami(token=hf_token)
    print(f"Logged in as: {user_info['name']}")
except Exception as e:
    print(f"Could not log in: {e}")
    print("Please make sure you have added your Hugging Face token to Colab Secrets with the name 'HF_TOKEN'")


Logged in as: Jeyas


## 🧠 Load the Summarization Model

In [3]:
from transformers import pipeline

MODEL_ID = "suriya7/bart-finetuned-text-summarization"
print(f"⏳ Loading model: {MODEL_ID}")

summarizer = pipeline(
    "summarization",
    model=MODEL_ID,
    tokenizer=MODEL_ID,
    device_map="auto"
)

print("✅ Model loaded successfully!")


⏳ Loading model: suriya7/bart-finetuned-text-summarization


config.json: 0.00B [00:00, ?B/s]

KeyError: "Unknown task summarization, available tasks are ['any-to-any', 'audio-classification', 'automatic-speech-recognition', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction', 'image-segmentation', 'image-text-to-text', 'image-to-image', 'keypoint-matching', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'table-question-answering', 'text-classification', 'text-generation', 'text-to-audio', 'text-to-speech', 'token-classification', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']"

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_ID = "suriya7/bart-finetuned-text-summarization"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID)

def generate_summary(text):
    inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_new_tokens=100, do_sample=False)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

text_to_summarize = """Now, there is no doubt that one of the most important aspects of any Pixel phone is its camera.
And there might be good news for all camera lovers. Rumours have suggested that the Pixel 9 could come with a telephoto lens,
improving its photography capabilities even further. Google will likely continue to focus on using AI to enhance its camera performance,
in order to make sure that Pixel phones remain top contenders in the world of mobile photography."""
summary = generate_summary(text_to_summarize)
print(summary)

Loading weights:   0%|          | 0/512 [00:00<?, ?it/s]

Google is rumoured to be about to unveil its next-generation Pixel smartphone, the Google Pixel 9, which is expected to come with a bigger screen and a new AI-powered camera, as well as a new range of sensors to improve the quality of the camera.


## ✨ Summarization Function

In [8]:
def summarize_text(text: str, max_len: int, min_len: int, beams: int):
    if not text or len(text.strip()) < 30:
        return "⚠️ Please enter at least 30 characters.", {}

    start = time.time()

    result = summarizer(
        text,
        max_length=int(max_len),
        min_length=int(min_len),
        num_beams=int(beams),
        do_sample=False,
        truncation=True
    )

    summary = result[0]["summary_text"]
    elapsed = round(time.time() - start, 3)

    meta = {
        "model_id": MODEL_ID,
        "input_chars": len(text_to_summarize),
        "summary_chars": len(summary),
        "runtime_seconds": elapsed,
        "timestamp": datetime.utcnow().isoformat() + "Z",
        "params": {
            "max_length": int(max_len),
            "min_length": int(min_len),
            "num_beams": int(beams),
        }
    }

    return summary, meta


## 🖥️ Gradio App

In [21]:
import gradio as gr

with gr.Blocks(theme=gr.themes.Soft(), title="HF Summarization Demo") as demo:
    gr.Markdown("# 🧠 Text Summarization Demo (Hugging Face + Gradio)")
    gr.Markdown(f"**Model:** `{MODEL_ID}`")

    text_input = gr.Textbox(lines=10, label="Input Text", placeholder="Paste your long text here...")

    #with gr.Accordion("Advanced Settings", open=False):
        #max_len = gr.Slider(50, 300, value=120, step=10, label="Max Summary Length")
        #min_len = gr.Slider(10, 150, value=30, step=5, label="Min Summary Length")
        #beams = gr.Slider(1, 8, value=4, step=1, label="Beam Search (num_beams)")

    summarize_btn = gr.Button("✨ Summarize", variant="primary")

    summary_output = gr.Textbox(lines=6, label="Summary")

    summarize_btn.click(
        fn=generate_summary,
        inputs=[text_input],
        outputs=[summary_output]
    )

demo.launch(share=False, debug=True)


/tmp/ipython-input-3529169837.py:3: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), title="HF Summarization Demo") as demo:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
